# Liberal Democrat constituency-level general election results
## By rank and by margin, 2010 to 2024

The purpose of this notebook is to determine the distribution of constituency-level results achieved by the Liberal Democrats in recent General Elections. I am particularly interested in calculating the rank order position in each constituency, the percentage of votes gained, and the number of votes required for the Liberal Democrats to win the seat at the following election. 

My hunch is that the 2019 General Election, while resulting in a reduced presence in Parliament, created a number of strong second party Lib Dem results that laid the foundations for the growth of first party results in 2024. Conversely, the 2024 election seems to have brought about the opposite result: a very small number of good second party finishes. I do not intend to submit these hunches to formally testing in this notebook. My ambition is only to clearly establish how the party's results have varied in recent elections, as a basis for formal hypothesis testing at a later date.

To help me create this notebook, I have used ChatGPT for the purposes of 1) interpreting error messages, and 2) to help with the syntax not covered in either Everything Counts or Data Science courses, in particular: a) using np.where() to conditionally substitute values in a column and b) for using lambda to lookup the values in a column.

In [4]:
import pandas as pd
import numpy as np
import requests

In [16]:
def wrangle(YY):
    ''' 
    Since 2015, the House of Commons library has published data containing General Election results per constituency. 
    The data are made available in csv files with a consistent file naming convention, and with consistent column names.
    The data cannot be downloaded directly via a URL, so this function presupposes that the user has downloaded the files and partyd in a folder called data.
    The function takes the final two digits of the year as an argument, and enters them into the filepath using an f-string.
    Since we only want to take the top five national parties plus the SNP and Plaid Cymru, it will take one of UKIP, The Brexit Party or Reform UK according to the year selected.
    The function then discards constituencies in Northern Ireland, which are contested by a different set of parties entirely.
    It also discards all columns other than the constituency name, first party, second party, and numeric results of the vote for the selected parties.
    Because the results columns contain numbers that contain commas, Python treats them as non-numeric data, so the function converts them into integers.
    So that validity checks can be easily run, the function creates an 'All Others' column that contains the value of all votes cast for parties other than those included.
    It then creates a set of columns containing the results for each party (and 'All Others') expressed as a percentage of the valid votes cast.
    
    '''

    df = pd.read_csv(f"data/HoC-GE20{YY}-results-by-constituency.csv") # opens the csv using the YY argument to complete the filepath


    # determine which far right party to include based on the year selected:

    if YY < 19:
        Far_Right = 'UKIP'
    elif YY == 19:
        Far_Right = 'BRX'
    else:
        Far_Right = 'RUK'
    
    df = df[df['Region name'] != 'Northern Ireland'] # removes constituencies in Northern Ireland

    df = df[df['First party'] != 'Spk'] # removes the constituency won by the Speaker (which is by convention not contested)
    

    # Remove all columns apart from those required:

    df = df[['Constituency name', 'First party', 'Second party', 'Valid votes', 'Con', 'Lab', 'LD', Far_Right, 'Green', 'SNP', 'PC']]


    # Insert underscores in party of spaces in column names:

    df.columns = df.columns.str.replace(' ', '_')
    

    # Add a column containing the year:

    df.insert(0, 'Year', f"20{YY}") 
    

    # Reformat the data in the columns containing the number of votes cast so that Python can treat the data as numeric:

    votes_columns = ['Valid_votes', 'Con', 'Lab', 'LD', Far_Right, 'Green', 'SNP', 'PC'] # creates a list variable for the columns to simplify the code
    

    # Create an 'Others' column    
    
    results_columns = votes_columns[1:] # create a list of columns containing the votes per party, excluding the 'Valid votes' column

    df['Others'] = df['Valid_votes'] - df[results_columns].sum(axis=1) # constructs an 'Others' column so we can ensure that the total votes cast sum to the correct number

    results_columns.append('Others') # add the new column to the list of results columns


    # Since parties grouped under 'Others' occasionally come first or second, update the 'First party' and 'Second party' columns accordingly, using np.where() code provided by ChatGPT:

    df['First_party'] = np.where(
      df['First_party'].isin(results_columns),
      df['First_party'],  # Keep the value if it's valid
     'Others'             # Replace with 'Others' if not valid
    )
    
    df['Second_party'] = np.where(
      df['Second_party'].isin(results_columns),
      df['Second_party'],  # Keep the value if it's valid
     'Others'             # Replace with 'Others' if not valid
    )


    # generate a set of columns containing the votes per party as a % of the valid votes cast, and store their names in a list:

    percent_columns = []

    for column in results_columns:
        df[f"{column}_%"] = df[column]/df['Valid_votes']*100
        percent_columns.append(f"{column}_%") # adds new columns to a list so we can run validity checks on these columns
        votes_columns.append(f"{column}_%") # also add them to the earlier votes_columns list, for the same purpose


    # generate columns containing Boolean validity checks on the resulting data:

    df['Percent_check'] = abs(100 - df[percent_columns].sum(axis = 1)) < 0.001 # checks that the percentages sum to a number close to 1

    print(df['Percent_check'].value_counts()) # print the number of True and False values to ascertain whether percentages correctly calculated

    df['Positive_check'] = (df[votes_columns] > 0).any(axis=1) # checks that all vote values are positive

    print(df['Positive_check'].value_counts()) # print the number of True and False values to ascertain whether all numeric values are positive

    # Create a column 'LD position' to store categorical assignment of constituences based on Lib Dem position in the results 

    df.insert(4, 'LD_position', df[results_columns].rank(axis=1, ascending=False)['LD']) # Create column between 'Second party' and 'Valid votes'
    df['LD_position'] = np.where( 
        df['LD_position'] == 1, '1st', # Change numeric value 1 to string value '1st'
        np.where(
            df['LD_position'] == 2, '2nd', # Change numeric value 2 to string value '2nd'
            np.where(
                df['LD_position'] == 3, '3rd', # Change numeric value 3 to string value '3rd'
                '4th or worse'  # Otherwise, replace value with string '4th or worse'
                )
            )
        )

    # Add new columns showing the number of votes won by the first and second place parties, using lambda code provided by ChatGPT:

    df.insert(3, 'First_party_votes', df.apply(lambda row: row[row['First_party']], axis=1))
    df.insert(5, 'Second_party_votes', df.apply(lambda row: row[row['Second_party']], axis=1))

    # Add new columns showing the percentage of votes won by the first and second place parties:

    df.insert(4, 'First_party_%', df['First_party_votes']/df['Valid_votes']*100)
    df.insert(7, 'Second_party_%', df['Second_party_votes']/df['Valid_votes']*100)

    # Add a new column showing the difference between the winning party's vote and the Lib Dems' vote, in percentage terms
    # Unless the Lib Dems are the winning party, in which case it should show a negative number measuring the difference between the Lib Dem vote and the second place vote 
              
    df.insert(9, 'LD_diff_%', '')
    
    df['LD_diff_%'] = np.where(
        df['First_party'] == 'LD', 
        df['LD_%'] - df['Second_party_%'],
        df['LD_%'] - df['First_party_%']
        )
    
    print(df.shape)  # print the number of columns and rows to enable visual checking for errors
    
    print(df.columns) # print the columns created to enable visual checking for errors

    print(df.describe()) # print the default descriptive statistics for each column to enable visual checking for errors 

    return df

Having created the above function, we now simply need to apply it to each dataset and ascertain that the validity check columns have produced True in every row, and visually check the .columns and .describe() outputs for any unexpected values.

In [17]:
GE2010 = wrangle(10)

Percent_check
True    631
Name: count, dtype: int64
Positive_check
True    631
Name: count, dtype: int64
(631, 29)
Index(['Year', 'Constituency_name', 'First_party', 'First_party_votes',
       'First_party_%', 'Second_party', 'Second_party_votes', 'Second_party_%',
       'LD_position', 'LD_diff_%', 'Valid_votes', 'Con', 'Lab', 'LD', 'UKIP',
       'Green', 'SNP', 'PC', 'Others', 'Con_%', 'Lab_%', 'LD_%', 'UKIP_%',
       'Green_%', 'SNP_%', 'PC_%', 'Others_%', 'Percent_check',
       'Positive_check'],
      dtype='object')
       First_party_votes  First_party_%  Second_party_votes  Second_party_%  \
count         631.000000     631.000000          631.000000      631.000000   
mean        21667.660856      47.087348        13206.640254       28.532539   
std          4792.330010       7.044061         4186.160664        7.212971   
min          6723.000000      29.357953         2061.000000        9.406495   
25%         17990.500000      41.740075        10319.000000       23.1176

In [18]:
GE2015 = wrangle(15)

Percent_check
True    631
Name: count, dtype: int64
Positive_check
True    631
Name: count, dtype: int64
(631, 29)
Index(['Year', 'Constituency_name', 'First_party', 'First_party_votes',
       'First_party_%', 'Second_party', 'Second_party_votes', 'Second_party_%',
       'LD_position', 'LD_diff_%', 'Valid_votes', 'Con', 'Lab', 'LD', 'UKIP',
       'Green', 'SNP', 'PC', 'Others', 'Con_%', 'Lab_%', 'LD_%', 'UKIP_%',
       'Green_%', 'SNP_%', 'PC_%', 'Others_%', 'Percent_check',
       'Positive_check'],
      dtype='object')
       First_party_votes  First_party_%  Second_party_votes  Second_party_%  \
count         631.000000     631.000000          631.000000      631.000000   
mean        23766.893819      49.976324        12172.264659       25.750039   
std          5213.694330       7.493106         4044.227242        7.751285   
min          8662.000000      30.956214         3445.000000        8.469206   
25%         19850.000000      43.963380         9037.500000       19.3637

In [19]:
GE2017 = wrangle(17)

Percent_check
True    631
Name: count, dtype: int64
Positive_check
True    631
Name: count, dtype: int64
(631, 29)
Index(['Year', 'Constituency_name', 'First_party', 'First_party_votes',
       'First_party_%', 'Second_party', 'Second_party_votes', 'Second_party_%',
       'LD_position', 'LD_diff_%', 'Valid_votes', 'Con', 'Lab', 'LD', 'UKIP',
       'Green', 'SNP', 'PC', 'Others', 'Con_%', 'Lab_%', 'LD_%', 'UKIP_%',
       'Green_%', 'SNP_%', 'PC_%', 'Others_%', 'Percent_check',
       'Positive_check'],
      dtype='object')
       First_party_votes  First_party_%  Second_party_votes  Second_party_%  \
count         631.000000     631.000000          631.000000      631.000000   
mean        27824.874802      55.847443        15738.378764       31.866021   
std          6488.832855       9.308433         4602.704138        8.516588   
min          6013.000000      29.227752         3355.000000        7.300938   
25%         23581.500000      49.671469        12669.500000       25.8869

In [20]:
GE2019 = wrangle(19)

Percent_check
True    631
Name: count, dtype: int64
Positive_check
True    631
Name: count, dtype: int64
(631, 29)
Index(['Year', 'Constituency_name', 'First_party', 'First_party_votes',
       'First_party_%', 'Second_party', 'Second_party_votes', 'Second_party_%',
       'LD_position', 'LD_diff_%', 'Valid_votes', 'Con', 'Lab', 'LD', 'BRX',
       'Green', 'SNP', 'PC', 'Others', 'Con_%', 'Lab_%', 'LD_%', 'BRX_%',
       'Green_%', 'SNP_%', 'PC_%', 'Others_%', 'Percent_check',
       'Positive_check'],
      dtype='object')
       First_party_votes  First_party_%  Second_party_votes  Second_party_%  \
count         631.000000     631.000000          631.000000      631.000000   
mean        27120.218700      54.593311        14459.554675       29.487187   
std          6578.896196       8.693520         4575.579353        8.462158   
min          6531.000000      34.647095         4018.000000        7.817917   
25%         21964.000000      48.174556        11279.500000       22.773765

In [21]:
GE2024 = wrangle(24)

Percent_check
True    631
Name: count, dtype: int64
Positive_check
True    631
Name: count, dtype: int64
(631, 29)
Index(['Year', 'Constituency_name', 'First_party', 'First_party_votes',
       'First_party_%', 'Second_party', 'Second_party_votes', 'Second_party_%',
       'LD_position', 'LD_diff_%', 'Valid_votes', 'Con', 'Lab', 'LD', 'RUK',
       'Green', 'SNP', 'PC', 'Others', 'Con_%', 'Lab_%', 'LD_%', 'RUK_%',
       'Green_%', 'SNP_%', 'PC_%', 'Others_%', 'Percent_check',
       'Positive_check'],
      dtype='object')
       First_party_votes  First_party_%  Second_party_votes  Second_party_%  \
count          631.00000     631.000000          631.000000      631.000000   
mean         18718.07607      42.399515        11724.936609       26.211794   
std           3583.34449       7.200862         3311.773649        5.832713   
min           6692.00000      26.721552         2856.000000       10.492225   
25%          16176.00000      36.733174         9192.000000       22.155180

Having standardised the five datasets, we now need to append them into a single master dataset for further analysis.